# Same Seed

In [25]:
import torch
import random
import numpy as np
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


same_seeds(48763)

# Read Image

In [26]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define custom dataset
class CustomImageDataset(Dataset):
    def __init__(self, adult_dir, children_dir, transform=None):
        self.adult_dir = adult_dir
        self.children_dir = children_dir
        self.transform = transform

        # Get all image file paths
        self.adult_images = [os.path.join(adult_dir, fname) for fname in os.listdir(adult_dir) if fname.endswith('.jpg')]
        self.children_images = [os.path.join(children_dir, fname) for fname in os.listdir(children_dir) if fname.endswith('.jpg')]

        # Combine adult and children image paths
        self.images = self.adult_images + self.children_images

        # Labels: adults = 0, children = 1
        self.labels = [0] * len(self.adult_images) + [1] * len(self.children_images)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

transform_train = transforms.Compose([
    transforms.Resize((256,256)),  # Resize images
    transforms.ToTensor(),          # Convert to tensor
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(20),      # Random rotation
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
    # transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),  # Random erasing
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize
])

# Image transformations
transform_test = transforms.Compose([
    transforms.Resize((256,256)),  # Resize images
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

batch_size = 16
# Dataset and DataLoader
adult_train_dir = 'dataset/train/adults'
children_train_dir = 'dataset/train/children'
adult_test_dir = 'dataset/test/adults'
children_test_dir = 'dataset/test/children'
dataset_train = CustomImageDataset(adult_train_dir, children_train_dir, transform=transform_train)
dataset_test = CustomImageDataset(adult_test_dir, children_test_dir, transform=transform_test)
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)



In [27]:
adult_demo_dir = 'dataset_demo/adults'
children_demo_dir = 'dataset_demo/children'
dataset_demo = CustomImageDataset(adult_demo_dir, children_demo_dir, transform=transform_test)
demo_loader = DataLoader(dataset_demo, batch_size=batch_size, shuffle=True)

# Neural Network

In [28]:
# import torch.nn as nn
# import torchvision

# class Network(nn.Module):
#     def __init__(self, num_classes=2):
#         super(Network , self).__init__()
#         self.model = torchvision.models.shufflenet_v2_x0_5(pretrained=True)
#         self.model = nn.Sequential(*list(self.model.children())[:-3])
#         self.dropout = nn.Dropout(p=0.02)
#         self.pool = nn.AdaptiveAvgPool2d((4,4))
#         self.fc1 = nn.Linear(in_features=96*4*4, out_features=num_classes)
#         # self.fc2 = nn.Linear(in_features=4, out_features=num_classes)

#     def forward(self, x):
#         x = self.model(x)
#         x = self.pool(x)
#         x = x.view(x.size(0), -1)
#         x = self.dropout(x)
        
#         x = self.fc1(x)
        
#         # x = self.fc2(x)
#         return x

import torch.nn as nn
import torchvision

class Network(nn.Module):
    def __init__(self, num_classes=2):
        super(Network , self).__init__()
        self.model = torchvision.models.shufflenet_v2_x0_5(pretrained=True)
        
        self.model = nn.Sequential(*list(self.model.children())[:-2])
        self.model.add_module('global_avg_pool', nn.AdaptiveAvgPool2d(1))
        self.model.add_module('flatten', nn.Flatten())
        self.model.add_module('dropout', nn.Dropout(p=0.2))
        self.model.add_module('fc', nn.Linear(in_features=192, out_features=2))
        

        self.dropout = nn.Dropout(p=0.02)
        self.fc = nn.Linear(in_features=2, out_features=num_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [34]:
from torchsummary import summary
from torchvision import transforms, models
from thop import profile
import copy
model = Network(num_classes=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)
#input_shape = (batch_size, 3, 224, 224)
#flops, macs, params = calculate_flops(model=model,input_shape=input_shape,output_as_string=True,output_precision=4)
#print("FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=0.0000005)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, min_lr=0.0000005)
random_tensor = torch.randn(1, 3, 256 ,256).to(device)
model_for_profiling = copy.deepcopy(model)

# Profile the cloned model without modifying the original model's state_dict
model_for_profiling.eval()  # Set the cloned model to evaluation mode
with torch.no_grad():
    flops, params = profile(model_for_profiling, inputs=(random_tensor,), verbose=False)
print(f"FLOPs: {flops}, Params: {params}")
summary(model,(3,256,256))
# 计算 FLOPs
# example_input = torch.randn(1, 3, 128, 128)
#flops = profile_macs(model, example_input)
#print(f"FLOPs: {flops}")


cuda
FLOPs: 44046916.0, Params: 143528.0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 128, 128]             648
       BatchNorm2d-2         [-1, 24, 128, 128]              48
              ReLU-3         [-1, 24, 128, 128]               0
         MaxPool2d-4           [-1, 24, 64, 64]               0
            Conv2d-5           [-1, 24, 32, 32]             216
       BatchNorm2d-6           [-1, 24, 32, 32]              48
            Conv2d-7           [-1, 24, 32, 32]             576
       BatchNorm2d-8           [-1, 24, 32, 32]              48
              ReLU-9           [-1, 24, 32, 32]               0
           Conv2d-10           [-1, 24, 64, 64]             576
      BatchNorm2d-11           [-1, 24, 64, 64]              48
             ReLU-12           [-1, 24, 64, 64]               0
           Conv2d-13           [-1, 24, 32, 32]             21

# Learning

In [30]:
# Training loop
num_epochs = 50
best_acc = 0.0

for epoch in range(num_epochs):
    # print learning rate
    # print(f'{scheduler.get_last_lr()[0]:.6f}')
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    correct_test = 0
    total_test = 0
    model.train()

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    scheduler.step(running_loss)

    model.eval()
    with torch.no_grad():
        for inputs, labels in demo_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    epoch_loss_train = running_loss / len(train_loader.dataset)
    epoch_acc_train = correct_train / total_train
    epoch_acc_test = correct_test / total_test

    if epoch_acc_test >= best_acc:
        best_acc = epoch_acc_test
        torch.save(model.state_dict(), 'best_model.pth')

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss_train:.4f}, Accuracy_train: {epoch_acc_train:.4f}, Accuracy_test: {epoch_acc_test:.4f}, Best Accuracy_test: {best_acc:.4f}')

print('Training complete')

Epoch 1/50, Loss: 0.6947, Accuracy_train: 0.4929, Accuracy_test: 0.5000, Best Accuracy_test: 0.5000
Epoch 2/50, Loss: 0.6905, Accuracy_train: 0.5375, Accuracy_test: 0.5250, Best Accuracy_test: 0.5250
Epoch 3/50, Loss: 0.6854, Accuracy_train: 0.5696, Accuracy_test: 0.6667, Best Accuracy_test: 0.6667
Epoch 4/50, Loss: 0.6752, Accuracy_train: 0.6357, Accuracy_test: 0.6917, Best Accuracy_test: 0.6917
Epoch 5/50, Loss: 0.6667, Accuracy_train: 0.6554, Accuracy_test: 0.6917, Best Accuracy_test: 0.6917
Epoch 6/50, Loss: 0.6503, Accuracy_train: 0.6964, Accuracy_test: 0.7000, Best Accuracy_test: 0.7000
Epoch 7/50, Loss: 0.6309, Accuracy_train: 0.7339, Accuracy_test: 0.7083, Best Accuracy_test: 0.7083
Epoch 8/50, Loss: 0.6103, Accuracy_train: 0.7536, Accuracy_test: 0.7333, Best Accuracy_test: 0.7333
Epoch 9/50, Loss: 0.5927, Accuracy_train: 0.7643, Accuracy_test: 0.7250, Best Accuracy_test: 0.7333
Epoch 10/50, Loss: 0.5715, Accuracy_train: 0.7839, Accuracy_test: 0.7333, Best Accuracy_test: 0.7333

# Testing ACC

In [31]:
model = Network(num_classes=2)
model.load_state_dict(torch.load('best_model.pth'))
model = model.to(device)
model.eval()

correct_test = 0
total_test = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

    epoch_acc_test = correct_test / total_test
    print(f'Accuracy_test: {epoch_acc_test:.4f}')

Accuracy_test: 0.8532


# Demo

In [33]:
model = Network(num_classes=2)
model.load_state_dict(torch.load('best_model.pth'))
model = model.to(device)
model.eval()

correct_test = 0
total_test = 0
with torch.no_grad():
    for inputs, labels in demo_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

    epoch_acc_test = correct_test / total_test
    print(f'Accuracy_test: {epoch_acc_test:.4f}')


Accuracy_test: 0.8333
